<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/66_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Developing interactive web apps with gradio and leafmap**

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install -U geemap gradio

In [ ]:
import ee
import gradio as gr
import geemap.foliumap as geemap

In [ ]:
Map = geemap.Map(center=[21.79, 70.87], zoom=3)
image = ee.Image("USGS/SRTMGL1_003")
vis_params = {
    "min": 0,
    "max": 6000,
    "palette": "terrain",
}
Map.addLayer(image, vis_params, "SRTM")
Map

Visualize an Earth Engine layer.

In [ ]:
def viz_dem(vmin, vmax, palette):
    Map = geemap.Map()
    image = ee.Image("USGS/SRTMGL1_003")
    vis_params = {
        "min": vmin,
        "max": vmax,
        "palette": palette,
    }
    Map.addLayer(image, vis_params, "SRTM")
    return Map.to_gradio()


vmin = gr.Number(value=0, label="Min value")
vmax = gr.Number(value=6000, label="Max value")
palette = gr.Textbox(value="terrain", label="Palette")
title = "Visualize Earth Engine Data"
demo = gr.Interface(viz_dem, [vmin, vmax, palette], "html", title=title)
demo.launch()

Visualize Earth Engine layers side by side.

In [ ]:
def split(left, right):
    Map = geemap.Map(center=(40, -100), zoom=4, height=600)

    nlcd_left = ee.Image(f"USGS/NLCD_RELEASES/2019_REL/NLCD/{left}").select("landcover")
    nlcd_right = ee.Image(f"USGS/NLCD_RELEASES/2019_REL/NLCD/{right}").select(
        "landcover"
    )

    left_layer = geemap.ee_tile_layer(nlcd_left, {}, f"NLCD {left}")
    right_layer = geemap.ee_tile_layer(nlcd_right, {}, f"NLCD {right}")

    Map.split_map(
        left_layer,
        right_layer,
    )
    return Map.to_gradio()


left_input = gr.Textbox(value="2001", label="Left Layer URL")
right_input = gr.Textbox(value="2019", label="Right Layer URL")

title = "Visualizing National Land Cover Database (NLCD)"
demo = gr.Interface(split, [left_input, right_input], "html", title=title)
demo.launch()

Visualize Cloud Optimized GeoTIFF (COG).

In [ ]:
def split(left, right):
    Map = geemap.Map(center=[21.79, 70.87], zoom=3)
    Map.split_map(left, right)
    print(Map.options["layersControl"])
    return Map.to_gradio()


left_url = (
    "https://github.com/opengeos/data/releases/download/raster/Libya-2023-07-01.tif"
)
right_url = (
    "https://github.com/opengeos/data/releases/download/raster/Libya-2023-09-13.tif"
)
left_input = gr.Textbox(value=left_url, label="Left Layer URL")
right_input = gr.Textbox(value=right_url, label="Right Layer URL")
title = "Visualze Cloud Optimized GeoTIFF (COG)"
demo = gr.Interface(split, [left_input, right_input], "html", title=title)
demo.launch()